<a href="https://colab.research.google.com/github/Ventiligo/EvgeniaFedorova2.github.io/blob/main/Life_is_Meme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4 pillow transformers python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.1/676.1 kB 11.9 MB/s eta 0:00:00


In [1]:
import requests
import sqlite3

conn = sqlite3.connect('memes.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS memes (
                    id TEXT PRIMARY KEY,
                    name TEXT,
                    url TEXT
                )''')

def fetch_memes():
    response = requests.get("https://api.imgflip.com/get_memes")
    data = response.json()

    if data['success']:
        memes = data['data']['memes']
        for meme in memes:
            cursor.execute("INSERT OR IGNORE INTO memes (id, name, url) VALUES (?, ?, ?)",
                           (meme['id'], meme['name'], meme['url']))
        conn.commit()
        print("Memes fetched and saved successfully!")
    else:
        print("Failed to fetch memes")

fetch_memes()


Memes fetched and saved successfully!


In [3]:
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2')

def generate_meme_text(prompt):
    generated_text = generator(prompt, max_length=30, num_return_sequences=1)[0]['generated_text']
    return generated_text.strip()

user_input = "When you forgot to submit"
print("Generated Meme Text:", generate_meme_text(user_input))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Meme Text: When you forgot to submit a comment, you can now do it in another language! See you soon.


In [4]:
from PIL import Image, ImageDraw, ImageFont
import random
import requests
from io import BytesIO

def get_random_meme():
    cursor.execute("SELECT url FROM memes ORDER BY RANDOM() LIMIT 1")
    return cursor.fetchone()[0]

def create_meme(image_url, text):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    width, height = image.size
    text_x = 10
    text_y = height - 60

    draw.text((text_x, text_y), text, font=font, fill="white")


    image.save("generated_meme.jpg")
    image.show()

random_meme_url = get_random_meme()
generated_text = generate_meme_text("When you realize it's Monday morning")
create_meme(random_meme_url, generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [14]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 5.2 MB/s eta 0:00:00


In [ ]:
import telebot
from telebot import types
from PIL import Image, ImageDraw, ImageFont
import random
import requests
from io import BytesIO
import sqlite3

TOKEN = "8055265842:AAG08Pwh--2UhFKGEwFPIV_JjGVMG8-PouM"
bot = telebot.TeleBot(TOKEN)

from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')

def get_random_meme():
    conn = sqlite3.connect('memes.db')
    cursor = conn.cursor()

    cursor.execute("SELECT url FROM memes ORDER BY RANDOM() LIMIT 1")
    meme_url = cursor.fetchone()[0]

    conn.close()
    return meme_url

def generate_meme_text(prompt):
    generated_text = generator(prompt, max_length=30, num_return_sequences=1)[0]['generated_text']
    return generated_text.strip()

def create_meme(image_url, text):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    draw = ImageDraw.Draw(image)

    try:
        font = ImageFont.truetype("arial.ttf", 40)
    except:
        font = ImageFont.load_default()

    width, height = image.size
    text_x = 10
    text_y = height - 100

    outline_color = "black"
    for offset in [(-2, -2), (-2, 2), (2, -2), (2, 2)]:
        draw.text((text_x + offset[0], text_y + offset[1]), text, font=font, fill=outline_color)

    draw.text((text_x, text_y), text, font=font, fill="white")

    image.save("generated_meme.jpg")

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Welcome! Send me a text, and I'll generate a meme for you!")

@bot.message_handler(func=lambda message: True)
def generate_meme_command(message):
    user_input = message.text
    generated_text = generate_meme_text(user_input)
    meme_url = get_random_meme()

    create_meme(meme_url, generated_text)

    with open("generated_meme.jpg", "rb") as meme_image:
        bot.send_photo(message.chat.id, meme_image, caption="Here's your meme!")

bot.polling()


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
